## Recipe Database!!

In [26]:
import pandas as pd
from sqlalchemy import create_engine

### 1. Read the CSV

In [27]:
df = pd.read_csv('recipes.csv')

In [29]:
df.head(2)

,Unnamed: 0,recipe_name,prep_time,cook_time,total_time,servings,yield,ingredients,directions,rating,url,cuisine_path,nutrition,timing,img_src
0,0,Apple-Cranberry Crostada,NaN,NaN,NaN,8,6 to 8 - servings,"3 tablespoons butter, 2 pounds Granny Smith ap...",Heat butter in a large skillet over medium-hig...,4.4,https://www.allrecipes.com/recipe/76931/apple-...,/Desserts/Fruit Desserts/Apple Dessert Recipes/,"Total Fat 18g 23%, Saturated Fat 7g 34%, Chole...","Servings: 8, Yield: 6 to 8 - servings",https://www.allrecipes.com/thmb/Tf1wH73bfH6Oql...
1,1,Apple Pie by Grandma Ople,30 mins,1 hrs,1 hrs 30 mins,8,1 9-inch pie,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 19g 24%, Saturated Fat 9g 46%, Chole...","Prep Time: 30 mins, Cook Time: 1 hrs, Total Ti...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...


### 2. Investigate the data (any nans, missing values, unneeded attributes)

In [31]:
# number of rows with nans
nan_rows = df[df.isna().any(axis=1)]
print("Number of rows with NaN values:\n", len(nan_rows))

# Filter out rows with NaN values
df_clean = df.dropna()
print("\nLength of clean df without NaN values:\n", len(df_clean))


Number of rows with NaN values:
 459

Length of clean df without NaN values:
 631


### 3. add recipe_id attribute (recplaced the "Unnanmed:0")

In [33]:
df_clean = df_clean.rename(columns={'Unnamed: 0': 'recipe_id'})
df_clean.head(5)

,recipe_id,recipe_name,prep_time,cook_time,total_time,servings,yield,ingredients,directions,rating,url,cuisine_path,nutrition,timing,img_src
1,1,Apple Pie by Grandma Ople,30 mins,1 hrs,1 hrs 30 mins,8,1 9-inch pie,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 19g 24%, Saturated Fat 9g 46%, Chole...","Prep Time: 30 mins, Cook Time: 1 hrs, Total Ti...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...
3,3,Apple Crisp,30 mins,45 mins,1 hrs 15 mins,12,1 9x13-inch pan,"10 cups all-purpose apples, peeled, cored and ...",Preheat the oven to 350 degrees F (175 degrees...,4.7,https://www.allrecipes.com/recipe/12409/apple-...,/Desserts/Crisps and Crumbles Recipes/Apple Cr...,"Total Fat 8g 11%, Saturated Fat 5g 25%, Choles...","Prep Time: 30 mins, Cook Time: 45 mins, Total ...",https://www.allrecipes.com/thmb/uAzhPOh86PfR-N...
4,4,Apple Pie Filling,20 mins,20 mins,2 hrs 40 mins,40,5 9-inch pies,"18 cups thinly sliced apples, 3 tablespoons le...",Toss apples with lemon juice in a large bowl a...,4.7,https://www.allrecipes.com/recipe/12681/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 0g 0%, Sodium 61mg 3%, Total Carbohy...","Prep Time: 20 mins, Cook Time: 20 mins, Additi...",https://www.allrecipes.com/thmb/c0bbYaS1V_mTt_...
7,7,Apple-Cranberry Crisp,25 mins,40 mins,1 hrs 5 mins,8,1 8-inch crisp,"2 pounds Granny Smith apples - peeled, cored a...",Preheat the oven to 375 degrees F (190 degrees...,4.6,https://www.allrecipes.com/recipe/41893/apple-...,/Desserts/Crisps and Crumbles Recipes/,"Total Fat 12g 15%, Saturated Fat 4g 22%, Chole...","Prep Time: 25 mins, Cook Time: 40 mins, Total ...",https://www.allrecipes.com/thmb/MjDDFmBWiEx9f1...
10,10,Apple Turnovers,30 mins,25 mins,55 mins,8,8 turnovers,"2 tablespoons lemon juice, 4 cups water, 4 Gr...",Combine the lemon and 4 cups water in a large ...,4.6,https://www.allrecipes.com/recipe/59124/apple-...,/Desserts/Fruit Desserts/Apple Dessert Recipes/,"Total Fat 26g 33%, Saturated Fat 8g 39%, Chole...","Prep Time: 30 mins, Cook Time: 25 mins, Total ...",https://www.allrecipes.com/thmb/6EpK8eOrDHd4MM...


### 4. Create a DB using sqlalchemy

In [36]:
from sqlalchemy import create_engine

# Create an SQLite engine
engine = create_engine('sqlite:///recipes.db')

# Save the DataFrame to a SQL table named "recipes"
df.to_sql('recipes', con=engine, if_exists='replace', index=False)


1090